In [2]:
from helpers.geometry import *
from helpers.sweep import *
import math

In [3]:
main()

x = 3.0, y = 2.0, obs = -1 and test:-1
x = 2.29, y = 4.12, obs = -1 and test:-1
x = 1.59, y = 6.24, obs = -1 and test:-1


IndexError: list index out of range